# PAMAP2 → TS2Vec → Fractal-SSL

Reproducible pipeline built on the refactored FractalSSL toolkit.

This notebook orchestrates the full experiment in four steps:

1. Cache sliding-window tensors from the raw PAMAP2 dataset.
2. Pre-train a TS2Vec backbone.
3. Fine-tune the backbone with the fractal self-supervised objective.
4. Evaluate representations with a logistic-regression probe.

All heavy lifting is delegated to the scripts under `../scripts/` so the same workflow can run locally or on a cluster.

In [ ]:
# Optional: install the dependencies inside the notebook runtime
# !pip install -r ../requirements.txt

In [ ]:
from pathlib import Path

PROJECT_ROOT = Path('..').resolve()
DATA_ROOT = Path('/path/to/PAMAP2_Dataset')
CACHE_DIR = PROJECT_ROOT / 'cache/pamap2'
CONFIG_PATH = PROJECT_ROOT / 'configs/pamap2_ts2vec_fractal.json'
BACKBONE_CKPT = PROJECT_ROOT / 'runs/pamap2/ts2vec/ts2vec_epoch050.pt'
FRACTAL_CKPT = PROJECT_ROOT / 'runs/pamap2/fractal/fractal_epoch100.pt'


In [ ]:
# Step 1 – preprocess the raw dataset into window caches
!python ../scripts/prepare_data.py pamap2 --data-root {DATA_ROOT} --cache-dir {CACHE_DIR}

In [ ]:
# Step 2 – TS2Vec backbone pretraining
!python ../scripts/train_ssl.py --config {CONFIG_PATH} --stage backbone

In [ ]:
# Step 3 – Fractal self-supervised fine-tuning
!python ../scripts/train_ssl.py --config {CONFIG_PATH} --stage fractal_ssl --backbone-checkpoint {BACKBONE_CKPT}

In [ ]:
# Step 4 – Linear probe evaluation
!python ../scripts/evaluate_linear.py --config {CONFIG_PATH} --checkpoint {FRACTAL_CKPT}